# Assignment "Segmenting and Clustering Neighborhoods in Toronto"

## Import standard libraries

In [257]:
import numpy as np
import pandas as pd

***
## PART ONE

**Webscaping Toronto data**

In [258]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = pd.read_html(url)

In [259]:
df = html_data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Adjust column names

In [260]:
df.rename(columns={"Postal Code": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace=True)

Ignore cells with a Borough that is Not assigned

In [261]:
df.Borough.replace('Not assigned',np.NaN, inplace=True)  
df=df[df['Borough'].notnull()]

If Neighborhood is Not assigned, then the neighborhood will be the same as the borough.

In [262]:
check = df['Neighborhood'] == 'Not assigned' 
check.value_counts()

False    103
Name: Neighborhood, dtype: int64

*Conclusion: 'Not assigned' Neighborhood does not occur*

In [263]:
# coding for if it occurs in future:
df2=df.copy()
df2.loc[df2['Neighborhood'] == "Not assigned", 'Neighborhood'] = df2['Borough']

The number of rows

In [264]:
df.reset_index(inplace=True, drop=True)
print('Number of rows: ', df.shape[0])
df.head()

Number of rows:  103


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


***
## PART TWO

**Add geographical coordinates**

In [57]:
!pip install pgeocode

In [265]:
import pgeocode

def get_location(postalcode):
  nomi = pgeocode.Nominatim('CA') # Country code CA for Canada
  location = nomi.query_postal_code(postalcode)
  return location.latitude, location.longitude 

In [ ]:
df[["Latitude","Longitude"]] = np.nan   # Add columns

for i in range(len(df)):
    df['Latitude'][i],df['Longitude'][i]=get_location(df.loc[i,'PostalCode'])

In [267]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


***
## PART THREE

**Explore and cluster the neighborhoods in Toronto, only boroughs that contain the word Toronto** 

Subframe of Neighbourhoods with Borough 'Toronto' 

In [268]:
df_toronto = df.copy()
df_toronto['Borough']= df_toronto['Borough'].apply(lambda x: x if ('Toronto' in x) else np.NaN) 

In [269]:
df_toronto=df_toronto[df_toronto['Borough'].notnull()] # Remove non relevant rows
df_toronto.reset_index(inplace=True, drop=True)

In [270]:
print('Number of rows: ', df_toronto.shape[0])
print('Different Toronto Boroughs:\n', df_toronto['Borough'].value_counts())
df_toronto.head()

Number of rows:  39
Different Toronto Boroughs:
 Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


### Map of Toronto with neighborhoods superimposed on top

Geograpical coordinates of Toronto

In [165]:
!pip install geopy

In [277]:
from geopy.geocoders import Nominatim

In [278]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Map of Toronto using latitude and longitude values

In [279]:
import folium 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(borough,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
  
map_toronto

### Explore neighborhoods in Toronto


Parameters

In [274]:
CLIENT_ID = '0EOJLBP4YWIRTOD0HYMGQY14MTDJWLUOGJJ1L40JRBQO1KBH' # your Foursquare ID Client Id:  
CLIENT_SECRET = 'STOI5OOLQKSWUYEIU1B2MJPA2UXS0XVAKDSISVPKGUML5JEK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500
neighborhood_latitude = df_toronto['Latitude'][0]
neighborhood_longitude =df_toronto['Longitude'][0]

Function to get nearby venues

In [275]:
import requests # library to handle requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Code to run the function on each neighborhood and create a new dataframe called `toronto_venues` with top 100 venues within a radius of 500 meters


In [276]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Resulting dataframe


In [280]:
print('Shape of the df :',toronto_venues.shape)
toronto_venues.sample(5)

Shape of the df : (1531, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1224,Stn A PO Boxes,43.6437,-79.3787,Roundhouse Park,43.641745,-79.384279,Park
908,"The Annex, North Midtown, Yorkville",43.6736,-79.4035,Madame Boeuf And Flea,43.675240,-79.406620,Burger Joint
1094,"Kensington Market, Chinatown, Grange Park",43.6541,-79.3978,Rasta Pasta,43.654207,-79.400469,Caribbean Restaurant
1436,"First Canadian Place, Underground city",43.6492,-79.3823,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
492,"Richmond, Adelaide, King",43.6496,-79.3833,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym


Let's check how many venues were returned for each neighborhood


In [281]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,94,94,94,94,94,94
"Brockton, Parkdale Village, Exhibition Place",39,39,39,39,39,39
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",13,13,13,13,13,13
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",57,57,57,57,57,57
Central Bay Street,57,57,57,57,57,57
Christie,12,12,12,12,12,12
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


Let's find out how many unique categories can be curated from all the returned venues


In [282]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


### Analyze Each Neighborhood


In [283]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,...,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [284]:
toronto_onehot.shape

(1531, 217)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [285]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,...,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.010638,0.000000,0.000000,0.010638,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.053191,0.000000,0.000000,0.000000,0.010638,0.000000,0.031915,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.000000,0.021277,0.000000,0.000000,0.021277,...,0.021277,0.0000,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.021277,0.010638,0.010638,0.0000,0.000000,0.000000,0.000000,0.00,0.010638,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.025641,0.000000,0.000000,0.025641,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.025641,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.051282,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,...,0.025641,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.025641,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.051282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.076923,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.035088,0.035088,0.052632,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000,0.000000,0.052632,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,...,0.017544,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.035088,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.017544,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0

New size


In [286]:
toronto_grouped.shape

(39, 217)

Let's print each neighborhood along with the top 5 most common venues


In [287]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.12
1              Bakery  0.05
2               Hotel  0.05
3  Seafood Restaurant  0.04
4                Café  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
           venue  freq
0     Restaurant  0.15
1    Coffee Shop  0.15
2    Yoga Studio  0.08
3      Bookstore  0.08
4  Deli / Bodega  0.08


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2  Italian Restaurant  0.05
3                 Bar  0.05
4              Bakery  0.04


----Central Bay Street----
                  

Let's put that into a pandas dataframe


First, let's write a function to sort the venues in descending order.


In [288]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [289]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Hotel,Café,Seafood Restaurant,Beer Bar,Cocktail Bar,Restaurant,Japanese Restaurant,Clothing Store
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Chiropractor,Japanese Restaurant,Boutique,Brewery,French Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Yoga Studio,Italian Restaurant,Breakfast Spot,Bookstore,Sushi Restaurant,Bank,Japanese Restaurant,Martial Arts School
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,Wings Joint,Restaurant,Park,Speakeasy,Grocery Store,French Restaurant
4,Central Bay Street,Coffee Shop,Café,Clothing Store,Bubble Tea Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Shoe Store,Shopping Mall,Spa


<a id='item4'></a>


### Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [290]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [313]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=101).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 0, 3, 3, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [315]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Thai Restaurant,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Spa,Food Truck
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Sushi Restaurant,Gym,Coffee Shop,College Cafeteria,Escape Room,Bubble Tea Shop,Burrito Place,Martial Arts School,Mexican Restaurant,Ethiopian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Hotel,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Tanning Salon,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,0,Café,Coffee Shop,Seafood Restaurant,Italian Restaurant,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Bakery,Hotel
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,0,Pub,Health Food Store,Trail,Cheese Shop,Gastropub,Bakery,Dessert Shop,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant


Finally, let's visualize the resulting clusters


In [316]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [317]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


### Examine Clusters


Determine the discriminating venue categories that distinguish each cluster

#### Cluster 1 Drinking and eating


In [319]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Thai Restaurant,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Spa,Food Truck
1,Downtown Toronto,0,Sushi Restaurant,Gym,Coffee Shop,College Cafeteria,Escape Room,Bubble Tea Shop,Burrito Place,Martial Arts School,Mexican Restaurant,Ethiopian Restaurant
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Hotel,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Tanning Salon,Ramen Restaurant
3,Downtown Toronto,0,Café,Coffee Shop,Seafood Restaurant,Italian Restaurant,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Bakery,Hotel
4,East Toronto,0,Pub,Health Food Store,Trail,Cheese Shop,Gastropub,Bakery,Dessert Shop,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant
5,Downtown Toronto,0,Coffee Shop,Bakery,Hotel,Café,Seafood Restaurant,Beer Bar,Cocktail Bar,Restaurant,Japanese Restaurant,Clothing Store
6,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Bubble Tea Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Shoe Store,Shopping Mall,Spa
7,Downtown Toronto,0,Café,Grocery Store,Playground,Baby Store,Coffee Shop,Park,Candy Store,Athletics & Sports,Escape Room,Ethiopian Restaurant
8,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Gym,Hotel,Salad Place,Steakhouse,Thai Restaurant,Asian Restaurant,American Restaurant
9,West Toronto,0,Park,Grocery Store,Furniture / Home Store,Bakery,Bank,Pet Store,Pharmacy,Pizza Place,Pool,Middle Eastern Restaurant


#### Cluster 2  Photography


In [320]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Photography Studio,Park,Wings Joint,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 3  Sports and playground


In [321]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Playground,Gym Pool,Park,Garden,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
29,Central Toronto,2,Tennis Court,Playground,Thai Restaurant,Gym,Grocery Store,Park,Escape Room,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
33,Downtown Toronto,2,Playground,Park,Grocery Store,Candy Store,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 4 Park

In [322]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,3,Park,Residential Building (Apartment / Condo),Wings Joint,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5 Home services


In [323]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Home Service,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
